# Merging Census 2021 and 2011 Data

## Overview 

This notebook contains code to create a dataset of UK census data for London to enable consistent comparison and analysis between 2011 and 2021 (as some LSOA and MSOA boundaries have change; borough boundaries remain unchanged). 

We utilise the ['exact fit lookup' file](https://hub.arcgis.com/datasets/ons::lsoa-2011-to-lsoa-2021-to-local-authority-district-2022-lookup-for-england-and-wales-version-2/about) between LSOAs. This lookup table has a _'change indicator'_ field that indicates which super output areas have changed between 2011 and 2021 (and how):
- **U**: _No Change from 2011 to 2021._ This means that direct comparisons can be made between these 2011 and 2021 LSOA.
- **S**: _Split._ This means that the 2011 LSOA has been split into two or more 2021 LSOA. There will be one record for each of the 2021 LSOA that the 2011 LSOA has been split into. This means direct comparisons can be made between estimates for the single 2011 LSOA and the estimates from the aggregated 2021 LSOA.
- **M**: _Merged._ 2011 LSOA have been merged with another one or more 2011 LSOA to form a single 2021 LSOA. This means direct comparisons can be made between the aggregated 2011 LSOAs’ estimates and the single 2021 LSOA’s estimates. 
- **X**: _The relationship between 2011 and 2021 LSOA is irregular and fragmented._ This has occurred where 2011 LSOA have been redesigned because of local authority district boundary changes, or to improve their social homogeneity. These can’t be easily mapped to equivalent 2021 LSOA like the regular splits (S) and merges (M), and therefore like for like comparisons of estimates for 2011 LSOA and 2021 LSOA are not possible. _Luckily there are none of these for London LSOAs!_

Merged 2011 LSOAs can accounted for easily: assuming the variable is a count we simply sum the values for the multiple 2011 LSOAs and apply to the new 2021 LSOA. 

2011 LSOAs that have been split into two or more 2021 LSOas are not so straight-forward and we need to make assumptions about the distribution of people/households across space. If we assume that populations are evenly distributed across space, we can use the areal proportion of the 2011 LSOA that the new (multiple) 2021 LSOAs contributed and scale 2011 data values by those proportions.  

The first part of the notebook is proof-of-concept, working through the required steps using just a single variable.  

The second part of the notebook then uses the proof-of-concept to merge multiple variables from 2011 and 2021 census into a single data file. 

### Required Libraries
The notebook assumes the `london-data` virtual environment has been activated

In [1]:
from datetime import date
print(f'Last tested: {date.today()}')

Last tested: 2023-09-04


In [2]:
import pyproj
import geopandas as gpd
import shapely
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

## Proof of Concept

### Overview

1. Read Data
    - Read exact fit lookup
        - subset to London Boroughs
    - Read 2011 data (e.g. KS101EWDATA06.CSV from [here](https://www.nomisweb.co.uk/census/2011/bulk/r2_2))
    - Read 2021 data (e.g. census2021-ts001-lsoa.csv from [here](https://www.nomisweb.co.uk/census/2021/bulk))
    - Read 2021 geometries (london-2021-lsoa.shp created [here](https://github.com/jamesdamillington/london-data/blob/main/code/london-census2021-geography.ipynb))
    - calculate polygon area
2. Merge Data
    - Merge 2011 data to bestfit on LSOA11CD 
    - Merge 2021 data to bestfit on LSOA21CD
    - Merge 2021 areas on LSOA21CD
3. Calculate Areal Proportions
    - Create '2021 area prop' column:
        - Where CHGIND is U, set value 1
        - Where CHGIND is M, set value 2
        - Where CHGIND is S: calculate proportion from sum of areas for LSOAs with identical LSOA11CD 
4. Set Values
    - Create 'merged 2011' column
        - Where CHGIND is U, use original 2011 data value 
        - Where CHGIND is M, use sum of 2011 LSOAs with the LSOA21CD for M
        - Where CHGIND is S, multiply original 2011 data value by '2021 area prop'

### Read Data

Read exact fit lookup spreadsheet then subset to London Boroughs

In [3]:
census_igpath = Path("../data/inputs/geographies/census/")
efit = pd.read_csv(census_igpath / "Lookup-ExactFit-LSOA11_to_LSOA21_to_LAD22_EW_Version_2.csv",
                      usecols = list(range(7)))

In [4]:
efit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35796 entries, 0 to 35795
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   LSOA11CD  35796 non-null  object
 1   LSOA11NM  35796 non-null  object
 2   LSOA21CD  35796 non-null  object
 3   LSOA21NM  35796 non-null  object
 4   CHGIND    35796 non-null  object
 5   LAD22CD   35796 non-null  object
 6   LAD22NM   35796 non-null  object
dtypes: object(7)
memory usage: 1.9+ MB


In [5]:
lads = pd.unique(efit.LAD22NM)
lads

array(['City of London', 'Barking and Dagenham', 'Bexley', 'Barnet',
       'Brent', 'Bromley', 'Croydon', 'Camden', 'Ealing', 'Enfield',
       'Greenwich', 'Hackney', 'Haringey', 'Hammersmith and Fulham',
       'Hillingdon', 'Harrow', 'Havering', 'Islington', 'Hounslow',
       'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth',
       'Lewisham', 'Merton', 'Redbridge', 'Newham', 'Sutton', 'Southwark',
       'Richmond upon Thames', 'Tower Hamlets', 'Waltham Forest',
       'Wandsworth', 'Bury', 'Bolton', 'Westminster', 'Manchester',
       'Oldham', 'Rochdale', 'Salford', 'Stockport', 'Trafford', 'Wigan',
       'Tameside', 'Knowsley', 'Liverpool', 'Sefton', 'Wirral',
       'St. Helens', 'Doncaster', 'Barnsley', 'Rotherham', 'Sheffield',
       'Gateshead', 'Newcastle upon Tyne', 'North Tyneside', 'Sunderland',
       'South Tyneside', 'Birmingham', 'Coventry', 'Dudley', 'Sandwell',
       'Solihull', 'Walsall', 'Wolverhampton', 'Bradford', 'Calderdale',
       'Leeds', '

From this we can see London boroughs _are_ listed first (except for Westminster - Bury and Bolton will need to be dropped), so we can get list of borough names relatively easily:

In [6]:
lads = list(lads)
london_lads = lads[:lads.index('Westminster')+1]
london_lads.remove("Bury")
london_lads.remove("Bolton")
print(london_lads)
print(len(london_lads))

['City of London', 'Barking and Dagenham', 'Bexley', 'Barnet', 'Brent', 'Bromley', 'Croydon', 'Camden', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Haringey', 'Hammersmith and Fulham', 'Hillingdon', 'Harrow', 'Havering', 'Islington', 'Hounslow', 'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Redbridge', 'Newham', 'Sutton', 'Southwark', 'Richmond upon Thames', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']
33


Now subset using this list of 33 boroughs

In [7]:
efit = efit[efit['LSOA21NM'].str.contains("|".join(london_lads))]   #from https://stackoverflow.com/a/71399966
efit = efit.copy(deep=False)
efit.head()

,LSOA11CD,LSOA11NM,LSOA21CD,LSOA21NM,CHGIND,LAD22CD,LAD22NM
0,E01000001,City of London 001A,E01000001,City of London 001A,U,E09000001,City of London
1,E01000002,City of London 001B,E01000002,City of London 001B,U,E09000001,City of London
2,E01000003,City of London 001C,E01000003,City of London 001C,U,E09000001,City of London
3,E01000005,City of London 001E,E01000005,City of London 001E,U,E09000001,City of London
4,E01000006,Barking and Dagenham 016A,E01000006,Barking and Dagenham 016A,U,E09000002,Barking and Dagenham


In [8]:
pd.unique(efit.LAD22NM)

array(['City of London', 'Barking and Dagenham', 'Bexley', 'Barnet',
       'Brent', 'Bromley', 'Croydon', 'Camden', 'Ealing', 'Enfield',
       'Greenwich', 'Hackney', 'Haringey', 'Hammersmith and Fulham',
       'Hillingdon', 'Harrow', 'Havering', 'Islington', 'Hounslow',
       'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth',
       'Lewisham', 'Merton', 'Redbridge', 'Newham', 'Sutton', 'Southwark',
       'Richmond upon Thames', 'Tower Hamlets', 'Waltham Forest',
       'Wandsworth', 'Westminster', 'Brentwood'], dtype=object)

We end up including Brentwood (not a London borough) so need to remove this

In [9]:
efit = efit[~efit['LSOA21NM'].str.contains('Brentwood')]

In [10]:
pd.unique(efit.LAD22NM)

array(['City of London', 'Barking and Dagenham', 'Bexley', 'Barnet',
       'Brent', 'Bromley', 'Croydon', 'Camden', 'Ealing', 'Enfield',
       'Greenwich', 'Hackney', 'Haringey', 'Hammersmith and Fulham',
       'Hillingdon', 'Harrow', 'Havering', 'Islington', 'Hounslow',
       'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth',
       'Lewisham', 'Merton', 'Redbridge', 'Newham', 'Sutton', 'Southwark',
       'Richmond upon Thames', 'Tower Hamlets', 'Waltham Forest',
       'Wandsworth', 'Westminster'], dtype=object)

This list of unique boroughs in our bestfit table now looks right.

In [11]:
efit.to_csv(census_igpath / "Lookup-ExactFit-LSOA11_to_LSOA21_to_LAD22_EW_Version_2-London.csv", 
            index=False)

Next, read 2011 data 

In [12]:
census_icpath = Path("../data/inputs/census/")

In [13]:
ts11 = pd.read_csv(census_icpath / "KS101EWDATA06.CSV")

In [14]:
ts11.head()

,GeographyCode,KS101EW0001,KS101EW0002,KS101EW0003,KS101EW0004,KS101EW0005,KS101EW0006,KS101EW0007,KS101EW0008,KS101EW0009,KS101EW0010,KS101EW0011,KS101EW0012
0,E01000001,1465,767,698,1465,0,21,12.98,112.865948,52.354949,47.645051,100.000000,0.000000
1,E01000002,1436,767,669,1436,0,22,22.84,62.872154,53.412256,46.587744,100.000000,0.000000
2,E01000003,1346,714,632,1250,96,12,5.91,227.749577,53.046062,46.953938,92.867756,7.132244
3,E01000005,985,528,457,985,0,5,18.96,51.951477,53.604061,46.395939,100.000000,0.000000
4,E01000006,1703,866,837,1699,4,16,14.66,116.166439,50.851439,49.148561,99.765120,0.234880


In [15]:
ts11 = ts11[['GeographyCode', 'KS101EW0001']]
ts11 = ts11.set_axis(['LSOA11CD', 'TotalRes11'], axis=1)

In [16]:
ts11.head()

,LSOA11CD,TotalRes11
0,E01000001,1465
1,E01000002,1436
2,E01000003,1346
3,E01000005,985
4,E01000006,1703


Read 2021 data

In [17]:
ts21 = pd.read_csv(census_icpath / "census2021-ts001-lsoa.csv")

In [18]:
ts21.head()

,date,geography,geography code,Residence type: Total; measures: Value,Residence type: Lives in a household; measures: Value,Residence type: Lives in a communal establishment; measures: Value
0,2021,Hartlepool 001A,E01011954,2284,2284,0
1,2021,Hartlepool 001B,E01011969,1344,1344,0
2,2021,Hartlepool 001C,E01011970,1070,1070,0
3,2021,Hartlepool 001D,E01011971,1323,1323,0
4,2021,Hartlepool 001F,E01033465,1955,1955,0


In [19]:
ts21 = ts21[['geography code', 'Residence type: Total; measures: Value']]
ts21 = ts21.set_axis(['LSOA21CD', 'TotalRes21'], axis=1)

In [20]:
ts21.head()

,LSOA21CD,TotalRes21
0,E01011954,2284
1,E01011969,1344
2,E01011970,1070
3,E01011971,1323
4,E01033465,1955


Read 2021 geometries then calculate polygon area

In [21]:
census_gpath = Path("../data/geographies/census/")
boundaries = gpd.read_file(census_gpath / "london-2021-lsoa.shp").set_index('LSOA21NM')

In [22]:
print(boundaries.crs)

epsg:27700


In [23]:
boundaries['sqkm21'] = round(boundaries['geometry'].area / 10**6, 5)

In [24]:
boundaries.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4994 entries, City of London 001A to Westminster 024G
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   LSOA21CD  4994 non-null   object  
 1   MSOA21CD  4994 non-null   object  
 2   MSOA21NM  4994 non-null   object  
 3   LAD22CD   4994 non-null   object  
 4   LAD22NM   4994 non-null   object  
 5   geometry  4994 non-null   geometry
 6   sqkm21    4994 non-null   float64 
dtypes: float64(1), geometry(1), object(5)
memory usage: 312.1+ KB


In [25]:
areas = boundaries[['LSOA21CD', 'sqkm21']]

Note that we have dropped the geometry column here. The full resolution (highest precision) geometry data produces a large (~90MB) file so we drop here and will merge simplified geometry (_london-2021-lsoa-gen20.csv_ from `london-census2021-geography.ipynb`) later.

### Merge Data
- Merge 2011 data to bestfit on LSOA11CD 
- Merge 2021 data to bestfit on LSOA21CD
- Merge 2021 areas on LSOA21CD

In [26]:
merge_efit = pd.merge(efit, ts11, how='left', on='LSOA11CD')

In [27]:
merge_efit = pd.merge(merge_efit, ts21, how='left', on='LSOA21CD')

In [28]:
merge_efit = pd.merge(merge_efit, areas, how='left', on='LSOA21CD')

In [29]:
merge_efit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5016 entries, 0 to 5015
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LSOA11CD    5016 non-null   object 
 1   LSOA11NM    5016 non-null   object 
 2   LSOA21CD    5016 non-null   object 
 3   LSOA21NM    5016 non-null   object 
 4   CHGIND      5016 non-null   object 
 5   LAD22CD     5016 non-null   object 
 6   LAD22NM     5016 non-null   object 
 7   TotalRes11  5016 non-null   int64  
 8   TotalRes21  5016 non-null   int64  
 9   sqkm21      5016 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 431.1+ KB


### Calculate Proportions
- Create '2021 area prop' column:
    - Where CHGIND is U, set value 1
    - Where CHGIND is M, set value 2
    - Where CHGIND is S: calculate proportion from sum of areas for LSOAs with identical LSOA11CD 

For U and S, groupby 2011 LSOA code and calculate the proportion (this will be 1 for unsplit LSOAs) 

In [30]:
merge_efit['prop21'] = round(merge_efit['sqkm21'] / merge_efit.groupby('LSOA11CD')['sqkm21'].transform('sum'),4)
#from https://stackoverflow.com/a/57359372

In [31]:
merge_efit.loc[merge_efit['CHGIND'] == 'M', 'prop21'] = 2

### Set Values
- Create 'merged 2011' column
    - Where CHGIND is U, use original 2011 data value 
    - Where CHGIND is M, use sum of 2011 LSOAs with the LSOA21CD for M
    - Where CHGIND is S, multiply original 2011 data value by '2021 area prop'

In [32]:
merge_efit['TotalRes11_21c'] = round(merge_efit['TotalRes11'] * merge_efit['prop21'])
merge_efit['TotalRes11_21c'] = merge_efit['TotalRes11_21c'].astype(int)

In [33]:
merge_efit.loc[merge_efit['CHGIND']=='M','TotalRes11_21c'] = merge_efit[merge_efit['CHGIND']=='M'].groupby('LSOA21CD')['TotalRes11'].transform('sum')

In [34]:
merge_efit[merge_efit['CHGIND']=='S'].head()

,LSOA11CD,LSOA11NM,LSOA21CD,LSOA21NM,CHGIND,LAD22CD,LAD22NM,TotalRes11,TotalRes21,sqkm21,prop21,TotalRes11_21c
4606,E01003109,Lambeth 003E,E01033863,Lambeth 003G,S,E09000022,Lambeth,2255,1325,0.10111,0.3758,847
4607,E01033207,Lambeth 004F,E01033864,Lambeth 004H,S,E09000022,Lambeth,1708,1032,0.06158,0.2551,436
4608,E01033207,Lambeth 004F,E01033865,Lambeth 004I,S,E09000022,Lambeth,1708,1520,0.13870,0.5745,981
4609,E01033207,Lambeth 004F,E01033866,Lambeth 004J,S,E09000022,Lambeth,1708,1120,0.04116,0.1705,291
4610,E01003927,Southwark 002A,E01033867,Southwark 002F,S,E09000028,Southwark,2377,1547,0.16428,0.5815,1382


In [35]:
#with help from https://stackoverflow.com/a/68453949
cols = ['sqkm21','prop21','TotalRes21','TotalRes11','TotalRes11_21c']
merge_efit = merge_efit[[c for c in merge_efit.columns if c not in cols]+ cols]

In [36]:
merge_efit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5016 entries, 0 to 5015
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LSOA11CD        5016 non-null   object 
 1   LSOA11NM        5016 non-null   object 
 2   LSOA21CD        5016 non-null   object 
 3   LSOA21NM        5016 non-null   object 
 4   CHGIND          5016 non-null   object 
 5   LAD22CD         5016 non-null   object 
 6   LAD22NM         5016 non-null   object 
 7   sqkm21          5016 non-null   float64
 8   prop21          5016 non-null   float64
 9   TotalRes21      5016 non-null   int64  
 10  TotalRes11      5016 non-null   int64  
 11  TotalRes11_21c  5016 non-null   int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 509.4+ KB


In [30]:
census_opath = Path("../data/census/")
merge_efit.to_csv(census_opath / "merge_efit.csv", index=False)

## Multi-Variable Merge
With proof-of-concept established above, now combine code above to create a function to merge all remaining variables

In [63]:
#read meta file
metad = pd.read_csv(census_icpath / "london-census21-metadata.csv",
                    usecols=['Series', 'Variable_ids', 'Year', 'DataFile'])
metad.dropna(inplace=True)
metad = metad[~metad.Series.str.endswith('_21c')]
metad = metad.reset_index(drop=True)  #needed 'double loop' below
metad

,Series,Year,Variable_ids,DataFile
0,TotRes21,2021,Residence type: Total; measures: Value,census2021-ts001-lsoa.csv
1,TotRes11,2011,KS101EW0001,KS101EWDATA06.CSV
2,TotHHolds21,2021,Number of households: Number of households; me...,census2021-ts041-lsoa.csv
3,TotHHolds11,2011,QS406EW0001,QS406EWDATA06.CSV
4,FemRes21,2021,Sex: Female; measures: Value,census2021-ts008-lsoa.csv
5,FemRes11,2011,KS101EW0004,KS101EWDATA06.CSV
6,MalRes21,2021,Sex: Male; measures: Value,census2021-ts008-lsoa.csv
7,MalRes11,2011,KS101EW0003,KS101EWDATA06.CSV
8,Asian21,2021,"Ethnic group: Asian, Asian British or Asian Welsh",census2021-ts021-lsoa.csv
9,Asian11,2011,"KS201EW0010, KS201EW0011, KS201EW0012, KS201EW...",KS201EWDATA06.CSV


In [142]:
census_opath = Path("../data/census/")
dat_efit.to_csv(census_opath / "test_efit.csv", index=False)

In [143]:
dat_efit.tail()

,LSOA11CD,LSOA11NM,LSOA21NM,CHGIND,LAD22CD,LAD22NM,TotRes11,TotRes21,TotHHolds11,TotHHolds21,...,Asian11,Asian21,Black11,Black21,MixedE11,MixedE21,White11,White21,OtherE11,OtherE21
E01035720,E01004745,Westminster 021C,Westminster 021G,S,E09000033,Westminster,NaN,1218,NaN,588,...,NaN,134,NaN,84,NaN,129,NaN,797,NaN,73
E01035721,E01004666,Westminster 023B,Westminster 023H,M,E09000033,Westminster,NaN,2327,NaN,1307,...,NaN,420,NaN,271,NaN,126,NaN,1304,NaN,207
E01035721,E01004667,Westminster 023C,Westminster 023H,M,E09000033,Westminster,NaN,2327,NaN,1307,...,NaN,420,NaN,271,NaN,126,NaN,1304,NaN,207
E01035722,E01004738,Westminster 024D,Westminster 024G,M,E09000033,Westminster,NaN,2015,NaN,1180,...,NaN,444,NaN,45,NaN,88,NaN,1328,NaN,108
E01035722,E01004739,Westminster 024E,Westminster 024G,M,E09000033,Westminster,NaN,2015,NaN,1180,...,NaN,444,NaN,45,NaN,88,NaN,1328,NaN,108


In [35]:
import pyproj
import geopandas as gpd
import shapely
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

#MSOAs are missing!!

census_igpath = Path("../data/inputs/geographies/census/")
census_icpath = Path("../data/inputs/census/")
census_opath = Path("../data/census/")
census_gpath = Path("../data/geographies/census/")


boundaries = gpd.read_file(census_gpath / "london-2021-lsoa.shp").set_index('LSOA21NM')
boundaries['LSOA21KM2'] = round(boundaries['geometry'].area / 10**6, 5)
areas = boundaries[['LSOA21CD', 'LSOA21KM2']]

geometry = gpd.read_file(census_gpath / "london-2021-lsoa-gen20.shp")
geometry = geometry[['LSOA21CD', 'geometry']]

#read meta file
metad = pd.read_csv(census_icpath / "london-census21-metadata.csv",
                    usecols=['Series', 'Variable_ids', 'Year', 'DataFile'])
metad.dropna(inplace=True)
metad = metad[~metad.Series.str.endswith('_21c')]
metad = metad.reset_index(drop=True)  #needed 'double loop' below
metad


dat_efit = pd.read_csv(census_igpath / "Lookup-ExactFit-LSOA11_to_LSOA21_to_LAD22_EW_Version_2-London.csv")


def read_cdata(dpath, idf, yr):

    #in the two-row df, first row is 2021, second is 2011
    i = 0
    gc = 'geography code'
    if(yr == 2011):
        i = 1
        gc = 'GeographyCode'
    
    print(dpath / idf['DataFile'].iloc[i])  
    
    dat = pd.read_csv(dpath / idf['DataFile'].iloc[i])  #get DataFile name and read 
    
    vids = idf['Variable_ids'].iloc[i].split(",")   #this breaks for 2021 Asian and Black variables because of comma (work-around below)
    vids = [x.strip(' ') for x in vids]    #Remove spaces at the beginning and at the end of the string

    #annoyingly there are commas in column names for some variables, work-around here: 
    if(idf['Series'].iloc[i] == 'Asian21'):
        vids = ["Ethnic group: Asian, Asian British or Asian Welsh"]
    if(idf['Series'].iloc[i] == 'Black21'):
        vids = ["Ethnic group: Black, Black British, Black Welsh, Caribbean or African"]

    vids.insert(0, gc)
    #ts11 = ts11[[c for c in ts11.columns if c in vids]]  #safer than ts[vids] if column missing
    dat = dat[vids]
    
    #print(dat)
    dat = dat.set_index(gc)
    
    if dat.shape[i] > 1:
        dat[idf['Series'].iloc[i]] = dat.sum(axis=1)
        dat = dat.loc[:, [idf['Series'].iloc[i]]]   #return as a df, not Series
        
    else:
        dat = dat.rename(columns={vids[i]:idf['Series'].iloc[i]})
        
    dat = dat.reset_index()    #cannot use names='LSOA21CD' as version of pandas < 1.5 in london-data env
    if(yr == 2011):
        dat = dat.rename(columns={'GeographyCode':'LSOA11CD'})
    else:
        dat = dat.rename(columns={'geography code':'LSOA21CD'})

    return(dat)



for i, g in metad.groupby(metad.index // 2):
    
    dat21 = read_cdata(census_icpath, g, 2021)
    #print(dat21.info())
    dat11 = read_cdata(census_icpath, g, 2011)
    #print(dat11.info())
   
    dat_efit = pd.merge(dat_efit, dat11, how='left', on='LSOA11CD')  #need to merge here on LSOA11CD
    dat_efit = pd.merge(dat_efit, dat21, how='left', on='LSOA21CD')  #merge here on LSOA21CD


    
dat_efit = pd.merge(dat_efit, areas, how='left', on='LSOA21CD')
dat_efit['LSOAP21'] = round(dat_efit['LSOA21KM2'] / dat_efit.groupby('LSOA11CD')['LSOA21KM2'].transform('sum'),4)

dat_efit.loc[dat_efit['CHGIND'] == 'M', 'LSOAP21'] = 2

                       
for i, g in metad.groupby(metad.index // 2):

    newnm = g['Series'].iloc[1] + "_21c"
    dat_efit[newnm] = round(dat_efit[g['Series'].iloc[1]] * dat_efit['LSOAP21'])
    dat_efit.loc[dat_efit['CHGIND']=='M',newnm] = dat_efit[dat_efit['CHGIND']=='M'].groupby('LSOA21CD')[g['Series'].iloc[1]].transform('sum')

dat_efit = pd.merge(dat_efit, geometry, how='left', on='LSOA21CD')

#reorder columns for output  (with help from https://stackoverflow.com/a/53299403)
metad = pd.read_csv(census_icpath / "london-census21-metadata.csv",usecols=['Series'])
columns = metad['Series']
existing = [i for i in columns if i in dat_efit.columns]
dat_efit = dat_efit[existing]

#print(dat_efit.info())

dat_efit.to_csv(census_opath / "test_efit.csv", index=False)

    

../data/inputs/census/census2021-ts001-lsoa.csv
../data/inputs/census/KS101EWDATA06.CSV
../data/inputs/census/census2021-ts041-lsoa.csv
../data/inputs/census/QS406EWDATA06.CSV
../data/inputs/census/census2021-ts008-lsoa.csv
../data/inputs/census/KS101EWDATA06.CSV
../data/inputs/census/census2021-ts008-lsoa.csv
../data/inputs/census/KS101EWDATA06.CSV
../data/inputs/census/census2021-ts021-lsoa.csv
../data/inputs/census/KS201EWDATA06.CSV
../data/inputs/census/census2021-ts021-lsoa.csv
../data/inputs/census/KS201EWDATA06.CSV
../data/inputs/census/census2021-ts021-lsoa.csv
../data/inputs/census/KS201EWDATA06.CSV
../data/inputs/census/census2021-ts021-lsoa.csv
../data/inputs/census/KS201EWDATA06.CSV
../data/inputs/census/census2021-ts021-lsoa.csv
../data/inputs/census/KS201EWDATA06.CSV
../data/inputs/census/census2021-ts045-lsoa.csv
../data/inputs/census/QS416EWDATA06.CSV
../data/inputs/census/census2021-ts045-lsoa.csv
../data/inputs/census/QS416EWDATA06.CSV
../data/inputs/census/census2021

In [33]:
dat_efit.head()

,LSOA21CD,LSOA21NM,LAD22CD,LAD22NM,geometry,TotRes21,TotRes11,TotRes11_21c,TotHHolds21,TotHHolds11,...,Black11_21c,MixedE21,MixedE11,MixedE11_21c,White21,White11,White11_21c,OtherE21,OtherE11,OtherE11_21c
0,E01000001,City of London 001A,E09000001,City of London,"POLYGON ((532105.312 182010.574, 532162.491 18...",1475,1465,1465.0,838,876,...,11.0,56,54,54.0,1181,1238,1238.0,68,34,34.0
1,E01000002,City of London 001B,E09000001,City of London,"POLYGON ((532634.497 181926.016, 532619.141 18...",1384,1436,1436.0,824,830,...,4.0,60,54,54.0,1084,1274,1274.0,50,9,9.0
2,E01000003,City of London 001C,E09000001,City of London,"POLYGON ((532135.138 182198.131, 532158.250 18...",1613,1346,1346.0,1017,817,...,45.0,101,55,55.0,1195,1055,1055.0,107,23,23.0
3,E01000005,City of London 001E,E09000001,City of London,"POLYGON ((533808.018 180767.774, 533649.037 18...",1100,985,985.0,480,467,...,100.0,79,59,59.0,424,506,506.0,125,46,46.0
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,"POLYGON ((545122.049 184314.931, 545271.849 18...",1845,1703,1703.0,554,543,...,177.0,70,58,58.0,605,557,557.0,85,50,50.0


In [ ]:

    



for i, g in metad.groupby(metad.index // 2):
    
    #each iteration is a two-row df, first row for 2021, second for 2011
    
    #2021 first
    dat21 = pd.read_csv(census_icpath / g['DataFile'].iloc[0])
    
    dat21 = dat21[['geography code', 'Residence type: Total; measures: Value']]
    #ts21 = ts21.set_axis(['LSOA21CD', 'TotalRes21'], axis=1)
    
    
    print(census_icpath / g['DataFile'].iloc[1])  
    
    dat11 = pd.read_csv(census_icpath / g['DataFile'].iloc[1])  #get 2011 DataFile name and read 
    vids = g['Variable_ids'].iloc[1].split(",")
    vids = [x.strip(' ') for x in vids]
    vids.insert(0, 'GeographyCode')
    print(vids)
    #ts11 = ts11[[c for c in ts11.columns if c in vids]]  #safer than ts[vids] if column missing
    dat11 = dat11[vids]
    dat11 = dat11.set_index('GeographyCode')
    
    if dat11.shape[1] > 1:
        print(dat11.head())
        dat11[g['Series'].iloc[1]] = dat11.sum(axis=1)
        dat11 = dat11.loc[:, [g['Series'].iloc[1]]]   #return as a df, not Series
        print(dat11.head())
    
    else:
        dat11 = dat11.rename(columns={vids[1]:g['Series'].iloc[1]})
        print(dat11.head())



In [ ]:


ts11 = pd.read_csv(census_icpath / "KS101EWDATA06.CSV")
ts11 = ts11[['GeographyCode', 'KS101EW0001']]
ts11 = ts11.set_axis(['LSOA11CD', 'TotalRes11'], axis=1)

ts21 = pd.read_csv(census_icpath / "census2021-ts001-lsoa.csv")
ts21 = ts21[['geography code', 'Residence type: Total; measures: Value']]
ts21 = ts21.set_axis(['LSOA21CD', 'TotalRes21'], axis=1)

merge_efit = pd.merge(efit, ts11, how='left', on='LSOA11CD')
merge_efit = pd.merge(merge_efit, ts21, how='left', on='LSOA21CD')
merge_efit = pd.merge(merge_efit, areas, how='left', on='LSOA21CD')

merge_efit.loc[merge_efit['CHGIND'] == 'M', 'prop21'] = 2

merge_efit['TotalRes11_21c'] = round(merge_efit['TotalRes11'] * merge_efit['prop21'])
merge_efit['TotalRes11_21c'] = merge_efit['TotalRes11_21c'].astype(int)

merge_efit.loc[merge_efit['CHGIND']=='M','TotalRes11_21c'] = merge_efit[merge_efit['CHGIND']=='M'].groupby('LSOA21CD')['TotalRes11'].transform('sum')
merge_efit[merge_efit['CHGIND']=='S'].head()